# Erstellen Datengrundlage für ArcGIS Network Analyst
- Übernahme Daten aus OSM Protobuf statt Overture, da hier alle Tages enthalten sind
- Filtern auf relevante Netzwerkelemente
- Zuordnen von Parametern wie Ssperrung / Geschwinigkeit

In [ ]:
import duckdb
import geopandas as gpd
import requests
import os
import shutil
import time

## Download Pbf
- Geofabrik und lokale Ablage (zur Beschleunigung)
- nicht laufende Aktualisierung

In [ ]:
url = 'https://download.geofabrik.de/europe/germany/niedersachsen-latest.osm.pbf'
output_path = "downloads/niedersachsen-latest.osm.pbf"

In [ ]:
if os.path.exists(output_path):
    mtime = os.path.getmtime(output_path)
    age_hours = (time.time() - mtime) / 3600
    print(f"File '{output_path}' was last modified {age_hours:.2f} hours ago.")
else:
    print(f"File '{output_path}' does not exist.")

In [ ]:
# Download a pbf file to the downloads folder
# If the file is older than 72 hours, re-download it
if age_hours > 72:
    response = requests.get(url)
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded file to {output_path}")

## Initialisieren der DB (duckdb) und Einlesen des pbf Files
- https://duckdb.org/docs/stable/core_extensions/spatial/functions#st_readosm

### Verknüpfen der Nodes mit Koordinaten
- https://medium.com/data-science/how-to-read-osm-data-with-duckdb-ffeb15197390

In [ ]:
duck = duckdb.connect(database=':memory:') #als Inmory-Datenbank verbinden

In [ ]:
duck.sql("""INSTALL spatial;
            LOAD spatial;"""
         )

In [ ]:
duck.sql(f"DESCRIBE SELECT * FROM ST_READOSM('{output_path}');")

In [ ]:
duck.sql(f"SELECT * FROM ST_READOSM('{output_path}') where id = 68706379;") #68706379 Fußgängerzone vor Bremen Hbf

#### Erstellen einer Tabelle mit den relevanten Spalten

In [ ]:
duck.sql(f"""create or replace table matching_ways as 
            select 
            id, tags['name'] as name, 
            tags['highway'] as highway, tags['maxspeed'] as maxspeed, 
            tags['oneway'] as oneway, tags['oneway:bus'] as oneway_bus, 
            tags['area'] as area,  tags['access'] as access, tags['bus'] as bus,
            tags['layer'] as layer, tags['level'] as level,
            tags['lanes:bus'] as lanes_bus, tags['bus:lanes'] as bus_lanes,
            tags['lanes:psv'] as lanes_psv, tags['indoor'] as indoor, 
            refs
            
            from ST_READOSM('{output_path}')
            where tags['highway'] is not null and kind = 'way'
          -- LIMIT 500;         
         """)

In [ ]:
duck.sql(f"""
    CREATE OR REPLACE TEMP TABLE matching_ways_with_nodes_refs AS
        SELECT id, UNNEST(refs) as ref, UNNEST(range(length(refs))) as ref_idx
        FROM ST_READOSM('{output_path}')
        SEMI JOIN matching_ways USING (id)
        WHERE kind = 'way';

SELECT * FROM matching_ways_with_nodes_refs order by id, ref_idx;
         """)

In [ ]:
duck.sql(f"""
         CREATE or replace TEMP TABLE required_nodes_with_geometries AS
            SELECT id, ST_POINT(lon, lat) geometry
            FROM ST_READOSM('{output_path}') nodes
            SEMI JOIN matching_ways_with_nodes_refs
            ON nodes.id = matching_ways_with_nodes_refs.ref
            WHERE kind = 'node';

SELECT * FROM required_nodes_with_geometries;""")

In [ ]:
duck.sql("""CREATE or replace TEMP TABLE matching_ways_linestrings AS
SELECT
    matching_ways.id,
    matching_ways.name,
    matching_ways.highway,
    matching_ways.maxspeed,
    matching_ways.oneway,
    matching_ways.oneway_bus,
    matching_ways.area,
    matching_ways.access,
    matching_ways.layer,
    matching_ways.level,
    matching_ways.bus,
    matching_ways.lanes_bus,
    matching_ways.bus_lanes,
    matching_ways.lanes_psv,
    matching_ways.indoor,
    st_transform(ST_MakeLine(list(nodes.geometry ORDER BY ref_idx ASC)), 'EPSG:4326', 'EPSG:25832', always_xy:=true) linestring
FROM matching_ways
JOIN matching_ways_with_nodes_refs
ON matching_ways.id = matching_ways_with_nodes_refs.id
JOIN required_nodes_with_geometries nodes
ON matching_ways_with_nodes_refs.ref = nodes.id
GROUP BY 1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15;

SELECT * FROM matching_ways_linestrings where area = 'yes';""")

In [ ]:
duck.sql("""SELECT * 
         FROM matching_ways_linestrings 
         where area = 'yes' and highway = 'pedestrian' 
         -- and id = 68706379
         order by id;""")

#### Ermittlung der Strecken mit Busspur
- zunächst nur QSS

In [ ]:
duck.sql("""from matching_ways  
         where highway = 'busway'
         or bus_lanes is not null
         or lanes_bus is not null
         or lanes_psv is not null""")

### Parameter Bbox-Export und Auchluss Strecken

In [ ]:
bbox = '8.65 53.2, 9.05 52.95' # Bremen Südwest
highway_exclude = "'proposed', 'construction', 'bridleway', 'raceway', 'rest_area', 'services'"

### Anzeigen des Ausschnitts Bremen Südost inkl. Autobahn
- bei Anzeigeproblemen in VSCode mit Folium Help / Toggle Developers Tools

In [ ]:
# https://github.com/zvbn-code/osm-network-analyst/issues/1

gdf = gpd.GeoSeries.from_wkt(
    duck.sql(f"""
        SELECT ST_AsText(linestring) wkt, name, highway
        FROM matching_ways_linestrings
             where st_intersects(             
             ST_Transform(ST_Envelope('LINESTRING({bbox})'::geometry), 'EPSG:4326', 'EPSG:25832', always_xy:=true), linestring)
             and highway not in ({highway_exclude})
             
             -- limit 10000
    """).df()['wkt'],
    crs="EPSG:25832",
)
gdf.explore(
    tiles="CartoDB positron", 
    style_kwdsdict=dict(weight=1, 
                        cmap="Blues",
                        column='layer') , tooltip=True 
      )


### Ermitteln der Anzahl der Elemente und Ausgabe einer Datei für die Parameter Network Analyst
- Geschwindigkeit Bus
- gesperrt Bus / Fußweg

In [ ]:
duck.sql(f"""
    SELECT  highway, count(*) as count, 0 as bus_speed, false as bus_allowed, false as foot_allowed, false as bus
    FROM matching_ways_linestrings mw
            where st_intersects(             
            ST_Transform(ST_Envelope('LINESTRING({bbox})'::geometry), 'EPSG:4326', 'EPSG:25832', always_xy:=true), mw.linestring)
         and mw.highway not in ({highway_exclude})
         
    GROUP BY highway
         order by count desc
""").df().to_csv("highway_classify_raw.csv", index=False)

### Ermitteln der Fußgängerbereiche
- highway=pedestrian and area=yes

In [ ]:
gpd.GeoSeries.from_wkt(
    duck.sql("""
        SELECT ST_AsText(linestring) wkt
        FROM matching_ways_linestrings
             where st_intersects(             
             ST_Transform(ST_Envelope('LINESTRING(8.75 53.05, 8.9 53.1)'::geometry), 
             'EPSG:4326', 'EPSG:25832', always_xy:=true), linestring
             )
             and highway = 'pedestrian' and area = 'yes'
             
             -- limit 10000
    """).df()["wkt"],
    crs="EPSG:25832",
).explore(
    tiles="CartoDB positron", style_kwdsdict=dict(weight=1, cmap="Blues", column='layer')
) #.save

### Gekennzeichnete Wege für den Bus
- bus:lanes, lanes:bus, bus=yes, 

In [ ]:
gpd.GeoSeries.from_wkt(
    duck.sql("""
        SELECT ST_AsText(linestring) wkt
        FROM matching_ways_linestrings
             where st_intersects(             
             ST_Transform(ST_Envelope('LINESTRING(8.75 53.05, 8.9 53.1)'::geometry), 'EPSG:4326', 'EPSG:25832', always_xy:=true), linestring
             )
             and 
             (bus_lanes is not null or lanes_bus is not null or lanes_psv is not null or bus = 'yes')
             
             -- limit 10000
    """).df()["wkt"],
    crs="EPSG:25832",
).explore(
    tiles="CartoDB positron", style_kwdsdict=dict(weight=1, cmap="Blues", column='layer', tooltip=True)
) #.save

## Join Para and export GPKG Filegeodatabase

In [ ]:
duck.sql("""create or replace table para as select * from read_csv_auto('highway_classify.csv');""")
duck.sql("from para")

In [ ]:
duck.sql(f"""
    SELECT  mw.*, para.bus_speed, para.bus_allowed, para.foot_allowed, para.bus
    FROM matching_ways_linestrings mw
         
          LEFT JOIN para USING (highway)
            WHERE st_intersects(             
            ST_Transform(ST_Envelope('LINESTRING({bbox})'::geometry), 'EPSG:4326', 'EPSG:25832', always_xy:=true), mw.linestring)
            AND mw.highway not in ({highway_exclude})
         
""")

### Export to GeoPackage

In [ ]:
duck.sql(f"""copy(
    SELECT  mw.*, para.bus_speed, para.bus_allowed, para.foot_allowed, para.bus
    FROM matching_ways_linestrings mw
         
          LEFT JOIN para USING (highway)
            WHERE st_intersects(             
            ST_Transform(ST_Envelope('LINESTRING({bbox})'::geometry), 'EPSG:4326', 'EPSG:25832', always_xy:=true), mw.linestring)
            AND mw.highway not in ({highway_exclude})
            
            -- limit 10000
         
         ) to 'out/nds_hb_highways.gpkg' with (FORMAT 'GDAL', DRIVER 'GPKG', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES',  LAYER_NAME 'ausschnitt_hb', SRS 'EPSG:25832')
""")

### Export als FileGDB

- https://gdal.org/en/stable/drivers/vector/openfilegdb.html
- https://duckdb.org/docs/stable/core_extensions/spatial/gdal

In [ ]:
fgdb = 'out/nds_hb_highways.gdb'

In [ ]:
# Remove 'fgdb' folder if it exists
if os.path.exists(f'{fgdb}') and os.path.isdir(f'{fgdb}'):
    shutil.rmtree(f'{fgdb}')
    print("Removed 'fgdb' folder.")
else:
    print("'fgdb' folder does not exist.")

In [ ]:
duck.sql(f"""copy 
         (SELECT * 
         from matching_ways_linestrings 
         LEFT JOIN para USING (highway)
         where highway not in ('proposed', 'construction', 'bridleway', 'raceway', 'rest_area', 'services')
         -- limit 100 
         )
         to '{fgdb}' (  FORMAT gdal, 
                        DRIVER 'OpenFileGDB', 
                        SRS 'EPSG:25832', 
                        LAYER_NAME 'highways_nds_sel',
                        GEOMETRY_TYPE 'LINESTRING', 
                        LAYER_CREATION_OPTIONS (
                                                'FEATURE_DATASET=osm', 
                                                'LAYER_NAME=highways_nds_sel', 
                                                'LAYER_ALIAS=highways_nds_sel'
                                                ));""")

In [ ]:
duck.close()